In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline


In [3]:
df = pd.read_csv(r"C:\Users\mombo\OneDrive\Desktop\IRONHACK - COURSE\SEM 5\PROJECT\df_final_demo.txt")
df

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0
...,...,...,...,...,...,...,...,...,...
70604,7993686,4.0,56.0,38.5,U,3.0,1411062.68,5.0,5.0
70605,8981690,12.0,148.0,31.0,M,2.0,101867.07,6.0,6.0
70606,333913,16.0,198.0,61.5,F,2.0,40745.00,3.0,3.0
70607,1573142,21.0,255.0,68.0,M,3.0,475114.69,4.0,4.0


In [4]:
df1 = pd.read_csv(r"C:\Users\mombo\OneDrive\Desktop\IRONHACK - COURSE\SEM 5\PROJECT\df_final_experiment_clients.txt")

df1

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control
...,...,...
70604,2443347,NaN
70605,8788427,NaN
70606,266828,NaN
70607,1266421,NaN


In [5]:
# merge final_demo + final_experiment_clients
dem_ex=pd.merge(df, df1, on='client_id', how='left')

# drop NaN in column variation 
dem_ex.dropna(subset=['Variation','clnt_tenure_yr', 'clnt_tenure_mnth', 'num_accts', 'calls_6_mnth', 'logons_6_mnth'],inplace= True)

# Handle missing values
print("Missing values per column:\n", dem_ex.isnull().sum())

# column names in lower case
dem_ex.columns = dem_ex.columns.str.lower()

# columns conversions
cols_to_int = ['clnt_tenure_yr', 'clnt_tenure_mnth', 'num_accts', 'calls_6_mnth', 'logons_6_mnth']
dem_ex[cols_to_int] = dem_ex[cols_to_int].astype('Int64')

# drop duplicate from client_id
dem_ex.drop_duplicates(subset=['client_id'], keep='first', inplace=True)

# round column bal 
dem_ex['bal'] = dem_ex['bal'].round(2)

dem_ex

Missing values per column:
 client_id           0
clnt_tenure_yr      0
clnt_tenure_mnth    0
clnt_age            1
gendr               0
num_accts           0
bal                 0
calls_6_mnth        0
logons_6_mnth       0
Variation           0
dtype: int64


,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,variation
0,836976,6,73,60.5,U,2,45105.30,6,9,Test
1,2304905,7,94,58.0,U,2,110860.30,6,9,Control
2,1439522,5,64,32.0,U,2,52467.79,6,9,Test
3,1562045,16,198,49.0,M,2,67454.65,3,6,Test
4,5126305,12,145,33.0,F,2,103671.75,0,3,Control
...,...,...,...,...,...,...,...,...,...,...
50495,1780858,21,262,68.5,M,3,372100.59,6,9,Test
50496,6967120,21,260,68.5,M,3,4279873.38,6,9,Control
50497,5826160,20,249,56.5,F,2,44837.16,2,5,Test
50498,8739285,19,229,69.5,F,2,44994.24,1,4,Test


In [6]:
pt1 = pd.read_csv(r"C:\Users\mombo\OneDrive\Desktop\IRONHACK - COURSE\SEM 5\PROJECT\df_final_web_data_pt_1.txt", sep=',')
pt1

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
343136,2443347,465784886_73090545671,136329900_10529659391_316129,confirm,2017-03-31 15:15:46
343137,2443347,465784886_73090545671,136329900_10529659391_316129,step_3,2017-03-31 15:14:53
343138,2443347,465784886_73090545671,136329900_10529659391_316129,step_2,2017-03-31 15:12:08
343139,2443347,465784886_73090545671,136329900_10529659391_316129,step_1,2017-03-31 15:11:37


In [7]:
pt2 = pd.read_csv(r"C:\Users\mombo\OneDrive\Desktop\IRONHACK - COURSE\SEM 5\PROJECT\df_final_web_data_pt_2.txt", sep=',')
pt2

,client_id,visitor_id,visit_id,process_step,date_time
0,763412,601952081_10457207388,397475557_40440946728_419634,confirm,2017-06-06 08:56:00
1,6019349,442094451_91531546617,154620534_35331068705_522317,confirm,2017-06-01 11:59:27
2,6019349,442094451_91531546617,154620534_35331068705_522317,step_3,2017-06-01 11:58:48
3,6019349,442094451_91531546617,154620534_35331068705_522317,step_2,2017-06-01 11:58:08
4,6019349,442094451_91531546617,154620534_35331068705_522317,step_1,2017-06-01 11:57:58
...,...,...,...,...,...
412259,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
412260,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
412261,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
412262,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [8]:
# Combine the two web data files
web_data = pd.concat([pt1, pt2], axis=0, ignore_index=True)

# Drop duplicates
web_data.drop_duplicates(inplace=True)

# Handle missing values
print("Missing values per column:\n", web_data.isnull().sum())

# Drop rows missing key identifiers
web_data.dropna(subset=['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time'], inplace=True)

# Convert date column to datetime
web_data['date_time'] = pd.to_datetime(web_data['date_time'], errors='coerce')

# Drop rows where date_time failed conversion
web_data = web_data.dropna(subset=['date_time'])

# Ensure correct data types
web_data['client_id'] = web_data['client_id'].astype(int)

# Sort by client and time
web_data.sort_values(by=['client_id', 'date_time'], inplace=True)

# Reset index
web_data.reset_index(drop=True, inplace=True)

Missing values per column:
 client_id       0
visitor_id      0
visit_id        0
process_step    0
date_time       0
dtype: int64


In [9]:
# Merge demographics + experiment data and web data
merged = web_data.merge(dem_ex, on='client_id', how='left')
merged = merged.dropna()

merged.head()

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,variation
12,555,402506806_56087378777,637149525_38041617439_716659,start,2017-04-15 12:57:56,3,46,29.5,U,2,25454.66,2,6,Test
13,555,402506806_56087378777,637149525_38041617439_716659,step_1,2017-04-15 12:58:03,3,46,29.5,U,2,25454.66,2,6,Test
14,555,402506806_56087378777,637149525_38041617439_716659,step_2,2017-04-15 12:58:35,3,46,29.5,U,2,25454.66,2,6,Test
15,555,402506806_56087378777,637149525_38041617439_716659,step_3,2017-04-15 13:00:14,3,46,29.5,U,2,25454.66,2,6,Test
16,555,402506806_56087378777,637149525_38041617439_716659,confirm,2017-04-15 13:00:34,3,46,29.5,U,2,25454.66,2,6,Test


In [ ]:
# Calcul complition rate group TEST 

# number of unique visit which are started the process for group Test
df_start_test = merged[
    (merged['process_step'] == 'start') & 
    (merged['variation'] == 'Test')
]
N_start_test = df_start_test['visit_id'].nunique()

# number of unique visit which are  confirmed the process for group Test

df_confirm_test = merged[
    (merged['process_step'] == 'confirm') & 
    (merged['variation'] == 'Test')
]
N_confirm_test = df_confirm_test['visit_id'].nunique()


N_start_test, N_confirm_test



(33144, 21725)

In [ ]:
# Calcul of completion rate of TEST GROUP --> % of visits which started and completed for test group

completion_rate_test = (N_confirm_test / N_start_test) * 100

completion_rate_test

65.54730871349264

In [30]:
# Calcul complition rate group CONTROL

# number of unique visit which are started the process for group Control
df_start_control = merged[
    (merged['process_step'] == 'start') & 
    (merged['variation'] == 'Control')
]
N_start_control = df_start_control['visit_id'].nunique()

# number of unique visit which are  confirmed the process for group Control

df_confirm_control = merged[
    (merged['process_step'] == 'confirm') & 
    (merged['variation'] == 'Control')
]
N_confirm_control = df_confirm_control['visit_id'].nunique()


N_start_control, N_confirm_control

(30903, 16039)

In [37]:
# Calcul of completion rate of CONTROL GROUP --> % of visits which started and completed  for  control group
completion_rate_control =(N_confirm_control / N_start_control) * 100

completion_rate_control

51.901109924602785

Calculation of Time Spent on Each Step in GROUPS TEST AND CONTROL

In [ ]:
# conversion to datetime format for time calculations
merged['date_time'] = pd.to_datetime(merged['date_time'])

# Sort by visit_id and date_time to do calculation on the same visit session
merged = merged.sort_values(by=['visit_id', 'date_time'])

merged.head()

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,variation
262231,3561384,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:22:17,4,56,59.5,U,2,63130.44,6,9,Test
262232,3561384,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:23:09,4,56,59.5,U,2,63130.44,6,9,Test
546540,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:20:56,7,88,23.5,M,2,26436.73,6,9,Test
546541,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:12,7,88,23.5,M,2,26436.73,6,9,Test
546542,7338123,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:21:21,7,88,23.5,M,2,26436.73,6,9,Test


In [ ]:
# For each session, we calculate the difference between the current time and the time of the previous step (in a new column).
merged['time_diff'] = merged.groupby('visit_id')['date_time'].diff()
merged.head()

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,variation,time_diff
262231,3561384,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:22:17,4,56,59.5,U,2,63130.44,6,9,Test,NaT
262232,3561384,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:23:09,4,56,59.5,U,2,63130.44,6,9,Test,0 days 00:00:52
546540,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:20:56,7,88,23.5,M,2,26436.73,6,9,Test,NaT
546541,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:12,7,88,23.5,M,2,26436.73,6,9,Test,0 days 00:00:16
546542,7338123,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:21:21,7,88,23.5,M,2,26436.73,6,9,Test,0 days 00:00:09


In [ ]:
# For each session, we calculate the step duration (in a new column).
merged['step_duration'] = merged.groupby('visit_id')['time_diff'].shift(-1)

merged.head()

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,variation,time_diff,step_duration
262231,3561384,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:22:17,4,56,59.5,U,2,63130.44,6,9,Test,NaT,0 days 00:00:52
262232,3561384,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:23:09,4,56,59.5,U,2,63130.44,6,9,Test,0 days 00:00:52,NaT
546540,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:20:56,7,88,23.5,M,2,26436.73,6,9,Test,NaT,0 days 00:00:16
546541,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:12,7,88,23.5,M,2,26436.73,6,9,Test,0 days 00:00:16,0 days 00:00:09
546542,7338123,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:21:21,7,88,23.5,M,2,26436.73,6,9,Test,0 days 00:00:09,0 days 00:00:14


In [66]:


# Average time spent on each step by variation
average_time_spent = merged.groupby(['variation', 'process_step'])['step_duration'].mean()
average_time_spent_seconds = average_time_spent.dt.total_seconds().round(2)

#  step_order
step_order = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

#  indexation of columns 
merged_time = average_time_spent_seconds.reset_index()

#  Convert the 'process_step' column into an ordered categorical type
merged_time['process_step'] = pd.Categorical(
    merged_time['process_step'], 
    categories=step_order, 
    ordered=True)

# Sort the DataFrame by 'variation',then by the ordered order of 'process_step'
merged_time_sorted = merged_time.sort_values(by=['variation', 'process_step'])

merged_time_sorted




,variation,process_step,step_duration
1,Control,start,66.82
2,Control,step_1,50.55
3,Control,step_2,92.05
4,Control,step_3,137.21
0,Control,confirm,180.18
6,Test,start,61.47
7,Test,step_1,60.76
8,Test,step_2,88.88
9,Test,step_3,129.62
5,Test,confirm,250.57


Error Rates KPI